<a href="https://colab.research.google.com/github/Julio21-ai/NLP-GPO10/blob/main/5.2%20Vectores%20Embebidos%20Pre-entrenados/MNA_NLP_semana_5_Actividad_Embeddings_abril_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Evidence 3](https://i.imgur.com/mu6ZuGT.jpg)
# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Actividad Semana 5

### **Vectores Embebidos Pre-entrenados: Fasttext**

#### **Nombres y matrículas de los integrantes del equipo:**



*   **Julio Baltazar Colín: A01794476**
*   Elemento de lista



In [12]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords

import re
import string

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

# Instalar el paquete contractions
!pip install contractions
from collections import Counter
import nltk
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


##**Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [13]:
#carga de datos desde un repositorio externo:

import requests
urlAmzn = 'https://raw.githubusercontent.com/Julio21-ai/NLP-GPO10/main/5.2%20Vectores%20Embebidos%20Pre-entrenados/amazon5.txt'
urlimdb = 'https://raw.githubusercontent.com/Julio21-ai/NLP-GPO10/main/5.2%20Vectores%20Embebidos%20Pre-entrenados/imdb5.txt'
urlyelp = 'https://raw.githubusercontent.com/Julio21-ai/NLP-GPO10/main/3.2%20Actividad%202/yelp_labelled.txt'

responce_amzn = requests.get(urlAmzn)
responce_imdb = requests.get(urlimdb)
responce_urlyelp = requests.get(urlyelp)

data_amzn = None
data_imdb= None
data_urlyelp= None

if responce_amzn.status_code == 200: #200 = OK
    data_amzn = responce_amzn.text
else:
    print("Error al obtener el archivo:", responce_amzn.status_code)

if responce_imdb.status_code == 200: #200 = OK
    data_imdb = responce_imdb.text
else:
    print("Error al obtener el archivo:", responce_imdb.status_code)

if responce_urlyelp.status_code == 200: #200 = OK
    data_urlyelp = responce_urlyelp.text
else:
    print("Error al obtener el archivo:", responce_imdb.status_code)


In [14]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
from io import StringIO

dfa = pd.read_csv(StringIO(data_amzn), sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv(StringIO(data_imdb), delimiter='\t', names=['review','label'], header=None, encoding='utf-8')
dfy = pd.read_csv(StringIO(data_urlyelp), sep='\t', names=['review','label'], header=None, encoding='utf-8')


print('Total de registros de Amazon:',dfa.shape)
print('Total de registros de IMBD:',dfi.shape)
print('Total de registros de Yelp:',dfy.shape)

df = pd.concat([dfa, dfi, dfy], ignore_index=True)
print('Total de registros dataframe df:',df.shape)

# *********** Aquí termina la sección de agregar código *************


Total de registros de Amazon: (1000, 2)
Total de registros de IMBD: (1000, 2)
Total de registros de Yelp: (1000, 2)
Total de registros dataframe df: (3000, 2)


In [15]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  3000 non-null   object 
 1   label   2000 non-null   float64
dtypes: float64(1), object(1)
memory usage: 47.0+ KB


In [16]:
# Y veamos sus primeros registros:

df.head()

,review,label
0,So there is no way for me to plug it in here i...,0.0
1,"Good case, Excellent value.",1.0
2,Great for the jawbone.,1.0
3,Tied to charger for conversations lasting more...,0.0
4,The mic is great.,1.0


##**Pregunta - 2:**

Realiza el proceso de limpieza.

Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización.











In [17]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
def remover_espacios_extra(texto):
    # Eliminar los espacios iniciales y finales y dividir el texto por espacios
    texto_filtrado = re.sub(r'\s+', ' ', texto).strip().split()
    # texto_filtrado las palabras con un solo espacio
    retval = ' '.join(texto_filtrado)
    return retval

def procesar_comilla(texto):
    texto_filtrado = texto
    # Eliminar terminaciones 's
    texto_filtrado = re.sub(r"'s\b", '', texto_filtrado)
    # Reemplazar ".", "," , ":", y "_" por espacios en blanco
    texto_filtrado = re.sub(r"\'", ' ', texto_filtrado)
    return texto_filtrado

def eliminar_no_alfa(texto):
    texto_filtrado = texto

    # Eliminar caracteres no alfanuméricos, excepto ".", ",", ":", "-" y "_"
    texto_filtrado = re.sub(r"[^\w\s\.,:-]+", '', texto_filtrado)

    # Eliminar guiones repetidos en medio de la cadena
    texto_filtrado = re.sub(r'(?<!-)-{2,}(?!-)', ' ', texto_filtrado)

    # Eliminar guiones al principio y al final de la cadena
    texto_filtrado = re.sub(r'^-+|-+$', '', texto_filtrado)

    # Reemplazar ".", "," , ":", y "_" por espacios en blanco
    texto_filtrado = re.sub(r'[\.,:_]', ' ', texto_filtrado)

    return texto_filtrado

def minusculas(texto):
    #Convertir a minusculas
    return texto.lower()

def remover_caracteres_sueltos(texto):
    # Expresión regular para encontrar caracteres sueltos
    expresion_regular = r'\b\w{1}\b'  # Coincidirá con cualquier carácter único

    # Reemplaza los caracteres sueltos con una cadena vacía
    texto_filtrado = re.sub(expresion_regular, '', texto)

    return texto_filtrado

def eliminar_no_ascii(texto):
    # Eliminar caracteres no ASCII
    retval = re.sub(r'[^\x00-\x7F]+', '', texto)
    return  texto

def eliminar_digitos(texto):
    # Utilizar una expresión regular para eliminar palabras con dígitos
    texto_filtrado = re.sub(r'\b\w*\d\w*\b', '', texto)

    # Eliminar espacios adicionales resultantes
    #texto_filtrado = re.sub(r'\s+', ' ', texto_filtrado).strip()

    return texto_filtrado


def expandir_contracciones(texto):
    # Expandir contracciones
    return contractions.fix(texto)


def lematizado(texto):
    wnl = WordNetLemmatizer()
    # Tokenizar el texto en palabras
    palabras = word_tokenize(texto)

    palabras_lemmatizadas = []
    for palabra in palabras:
        # Lemmatizar la palabra como adjetivo
        lemmatizada = wnl.lemmatize(palabra, 'a')
        # Lemmatizar la palabra como verbo
        lemmatizada = wnl.lemmatize(lemmatizada, 'v')
        # Lemmatizar la palabra como sustantivo
        lemmatizada = wnl.lemmatize(lemmatizada, 'n')
        # Lemmatizar la palabra como adverbio
        lemmatizada = wnl.lemmatize(lemmatizada, 'r')
        palabras_lemmatizadas.append(lemmatizada)

    # Unir las palabras lemmatizadas en un texto de nuevo
    texto_filtrado = ' '.join(palabras_lemmatizadas)

    return texto_filtrado

def remove_stop_words(lista_palabras, stopwords):
    #Eliminar las stopwords
    palabras_sin_stopwords = [palabra for palabra in lista_palabras if palabra not in stopwords]
    return palabras_sin_stopwords


def tokenizar(enunciado):
    # Tokenizar el texto
    tokens = word_tokenize(enunciado)
    # Eliminar caracteres no alfanuméricos
    tokens = [token for token in tokens if token.isalnum()]
    return tokens

def limpiar_texto(texto):
  texto_filtrado = texto
  texto_filtrado = expandir_contracciones(texto_filtrado)
  texto_filtrado = procesar_comilla(texto_filtrado)
  texto_filtrado = eliminar_no_alfa(texto_filtrado)
  texto_filtrado = eliminar_no_ascii(texto_filtrado)
  texto_filtrado = eliminar_digitos(texto_filtrado)
  texto_filtrado = remover_espacios_extra(texto_filtrado)
  texto_filtrado = lematizado(texto_filtrado)
  texto_filtrado = remover_caracteres_sueltos(texto_filtrado)
  texto_filtrado = minusculas(texto_filtrado)
  return texto_filtrado

# *********** Aquí termina la sección de agregar código *************



In [18]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

X = df.review     # Serie de strings
Y = df.label      # Serie de enteros 0s y 1s


texto_limpio = X.apply(limpiar_texto)
df['review'].to_csv('rsultado_sucio.txt', index=False)
texto_limpio.to_csv('resultado_limpio.txt', index=False)

tokens_list = [tokenizar(texto) for texto in texto_limpio]
Xclean = tokens_list

# *********** Aquí termina la sección de agregar código *************

In [19]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[0:5]:
  print(x)


['so', 'there', 'be', 'no', 'way', 'for', 'me', 'to', 'plug', 'it', 'in', 'here', 'in', 'the', 'us', 'unless', 'go', 'by', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'for', 'the', 'jawbone']
['tied', 'to', 'charger', 'for', 'conversation', 'last', 'more', 'than', 'minute', 'major', 'problems']
['the', 'mic', 'be', 'great']


#**Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [20]:

# ************* Inicia la sección de agregar código:*****************************

from sklearn.model_selection import train_test_split

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)

print('X,y Train:', len(x_train), len(y_train))      # los "x_" son "list" y los "y_" son "Series"
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450
X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


#**Pregunta - 4:**



Construye tu vocabulario a continuación


In [22]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


lista_tokens = Counter()

for k in range(len(x_train)):
  lista_tokens.update(x_train[k])


# Contar la frecuencia de cada token
frecuencia_tokens = Counter(lista_tokens)

# Definir el umbral de frecuencia
umbral_frecuencia = 2

# Filtrar palabras con frecuencia menor al umbral
frecuencia_filtrada = {token: freq for token, freq in frecuencia_tokens.items() if freq >= umbral_frecuencia}

# Ordenar palabras por frecuencia
palabras_ordenadas = sorted(frecuencia_filtrada.items(), key=lambda item: item[1], reverse=True)
diccionario = dict(palabras_ordenadas)

# *********** Aquí termina la sección de agregar código *************

In [32]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')


# ******* Inicia la sección de agregar código: ***********
print(len(diccionario))

# *********** Aquí termina la sección de agregar código *************

Longitud del vocabulario generado:
1495


c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

La principal razon para usar el conjunto de entrenamiento para genrar el vocabulario e evitar el fitrado de información, evitando que los modelos tengan una ventaja al momento de realizar las predicciones. Esto evita el sesgo en la informacion y una mayor imparcialidad en el modelo.

Otra de las razones es la de mantener la consistencia al emplear un unico diccionario que seria comun para todos los modelos facilitando el manejo de la información y permitir que los datos sean reprodusibles.


### ++++++++ Termina la sección de agregar texto: +++++++++++


In [ ]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************


In [ ]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

#**Pregunta - 5:**


a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++


#**Pregunta - 6:**

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el “valor” será su vector embebido de dimensión 300.

Este diccionario deberá ser del mismo tamaño que el vocabulario previo que hayas construido previamente.

Es recomendable que una vez que generes el nuevo vocabulario de vectores embebidos, guardes dicho diccionario en un archivo.

Recuerda borrar la variable donde descargaste los 2 millones de vectores embebidos Fasttext.



In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

#**Pregunta - 7:**



Generamos los vectores embebidos a paertir de los conjuntos de entrenamiento, validación y preuba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

In [ ]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

#**Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [ ]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************


In [ ]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

#**Pregunta - 9:**



Reporte del mejor modelo.


In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

#**Pregunta - 10:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++

##**Fin de la Actividad de vectores Embebidos - FastText**